In [1]:
# Copyright (c) 2016-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree. An additional grant
# of patent rights can be found in the PATENTS file in the same directory.


import numpy as np
import time
from scipy.sparse import csr_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.neighbors import LSHForest
from sklearn.feature_extraction import DictVectorizer

In [2]:
# make sure you run 'python setup.py install' first!
import pysparnn

# Fetch data

In [3]:
dataset = fetch_20newsgroups(subset='all', shuffle=True)

docs = np.array([x.split() for x in dataset.data])
datas = np.array(range(len(docs)))

In [4]:
print 'Num docs: {}'.format(len(docs))
print 'Avg doc length: {}'.format(np.mean([len(x) for x in docs]))
words = set()
for doc in docs:
    words.update(doc)
print 'Num unique words: {}'.format(len(words))    

Num docs: 18846
Avg doc length: 283.656001273
Num unique words: 386410


# Build LSH & PySparNN indexes

In [5]:
class PySparNNTextSearch:
    def __init__(self, docs, datas):
        
        self.dv = DictVectorizer()
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        self.dv.fit(dicts)
        features = csr_matrix(self.dv.transform(dicts), dtype=int)
        self.cp = pysparnn.ClusterIndex(features, datas, 
                                        pysparnn.matrix_distance.UnitCosineDistance)
        
    def search(self, docs):
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        features = csr_matrix(self.dv.transform(dicts), dtype=int)
        return self.cp.search(features, return_distance=False, k=1, k_clusters=1, max_distance=0.05)
        

t0 = time.time()
snn_search = PySparNNTextSearch(docs, datas)
print(time.time() - t0)

26.0494530201


In [6]:
class LSHForrestSearch:
    def __init__(self, docs):
        self.lshf = LSHForest(n_estimators=1, n_candidates=1,
                     n_neighbors=1)
        self.dv = DictVectorizer()
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        self.dv.fit(dicts)
        features = self.dv.transform(dicts)
        # floats are faster
        # features = csr_matrix(features, dtype=int)
        self.lshf.fit(features)
        
    def search(self, docs):
        dicts = []
        for d in docs:
            dicts.append(dict([(w, 1) for w in d]))
        features = self.dv.transform(dicts)
        # floats are faster
        # features = csr_matrix(features, dtype=int)
        return self.lshf.kneighbors(features, return_distance=False)
    
t0 = time.time()    
lsh = LSHForrestSearch(docs) 
print(time.time() - t0)

12.3326971531


In [7]:
# code that will measure query time and accuracy
import time
import random

def accuracy(result, truth):
    ret =  []
    for r, t in zip(result, truth):
        ret.append(1 if t in r else 0)
    return np.array(ret)

def time_it(search_index, docs, query_index):
    # time how long the query takes
    t0 = time.time()
    neighbors = search_index.search(docs[query_index])
    delta = time.time() - t0

    return delta, accuracy(neighbors, query_index).mean()

def identity_benchmark(search_index, docs, n_trials=2000, docs_per_query=50):
    # Bootstrap code to measure the time and accuracy
    times = []
    accuracys = []
    for i in range(n_trials):
        query_index = random.sample(range(len(docs)), docs_per_query)
        time, accuracy = time_it(search_index, docs, query_index)
        time = time / docs_per_query
        times.append(time)
        accuracys.append(accuracy)
    return np.median(times), np.median(accuracys)

In [10]:
snn_time, snn_accuracy = identity_benchmark(snn_search, docs)
print('PySparNN median time per query: {0}'.format(snn_time)) 
print('PySparNN median accuracy: {0}'.format(snn_accuracy)) 

PySparNN median time per query: 0.00353950977325
PySparNN median accuracy: 1.0


In [12]:
lsh_time, lsh_accuracy = identity_benchmark(lsh, docs)
print('LSH median time per query: {0}'.format(lsh_time)) 
print('LSH median accuracy: {0}'.format(lsh_accuracy)) 

LSH median time per query: 0.00611482143402
LSH median accuracy: 1.0


In [13]:
lsh_time / snn_time

1.7275899279121745